In [10]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
import tkinter as tk
from tkinter import filedialog
from scipy.interpolate import make_interp_spline
import multiprocessing as mp
from datetime import datetime
import yaml
import shutil

def create_output_folder():
    """Create timestamped output folder for results"""
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_dir = f'model_evaluation_results_{timestamp}'
    os.makedirs(output_dir, exist_ok=True)
    print(f"\nCreated output directory: {output_dir}")
    return output_dir

def calculate_iou(box1, box2):
    """Calculate IoU between two boxes [x1, y1, x2, y2]"""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - intersection
    
    return intersection / union if union > 0 else 0

def load_class_descriptions(file_path):
    """Load class descriptions and filter for classes to use in model"""
    df = pd.read_csv(file_path)
    return df[df['UseInModel'] == True]

def load_and_filter_annotations(annotations_path, class_descriptions, image_dir, images_per_class=200):
    """Load and filter annotations based on available images and classes"""
    print("Loading annotations...")
    annotations_df = pd.read_csv(annotations_path)
    
    # Get valid class IDs
    valid_labels = class_descriptions['LabelName'].tolist()
    
    # Get available image IDs from the image directory
    available_images = {Path(f).stem for f in os.listdir(image_dir) 
                       if f.lower().endswith(('.jpg', '.jpeg', '.png'))}
    
    # Filter annotations for valid classes and available images
    filtered_df = annotations_df[
        (annotations_df['LabelName'].isin(valid_labels)) &
        (annotations_df['ImageID'].isin(available_images))
    ]
    
    # Select fixed number of images per class
    selected_images = set()
    class_image_dict = {}
    
    for class_id in valid_labels:
        # Get all images containing this class
        class_images = filtered_df[filtered_df['LabelName'] == class_id]['ImageID'].unique()
        
        # Ensure we have enough images
        if len(class_images) < images_per_class:
            print(f"Warning: Only {len(class_images)} images available for class {class_id}")
            selected = class_images
        else:
            # Randomly select fixed number of images
            selected = np.random.choice(class_images, images_per_class, replace=False)
        
        class_image_dict[class_id] = selected
        selected_images.update(selected)
    
    # Filter annotations to only include selected images
    final_df = filtered_df[filtered_df['ImageID'].isin(selected_images)].copy()
    
    print(f"Selected {len(selected_images)} unique images across all classes")
    for class_id in valid_labels:
        print(f"Class {class_id}: {len(class_image_dict[class_id])} images")
    
    return final_df, class_image_dict

def create_confidence_range():
    """Create a range of confidence thresholds with varying intervals"""
    conf_range = np.concatenate([
        np.arange(0.001, 0.01, 0.001),  # Fine sampling for 0.001-0.01
        np.arange(0.01, 0.03, 0.002),   # Medium sampling for 0.01-0.03
        np.arange(0.03, 0.1, 0.005),    # Coarser sampling for 0.03-0.1
        np.arange(0.1, 1.02, 0.02)      # Coarsest sampling for 0.1-1.0
    ])
    return np.unique(conf_range)

def process_single_image(args):
    """Process a single image and return its metrics"""
    print("process_single_image")
    try:
        image_path, model, confidence_range, ground_truth_df, target_class = args
        print(f"Processing {os.path.basename(image_path)}...")  # Progress tracking
        
        # Initialize results dictionary
        image_results = {conf: {
            'true_positives': 0,
            'false_positives': 0,
            'false_negatives': 0,
            'total_iou': 0,
            'total_predictions': 0,
            'confidence_scores': []
        } for conf in confidence_range}
        
        # Load image and get dimensions
        img = cv2.imread(image_path)
        if img is None:
            print(f"Failed to load image: {image_path}")
            return image_results
            
        height, width = img.shape[:2]
        
        # Get image ID and filter ground truth boxes
        image_id = Path(image_path).stem
        gt_boxes_df = ground_truth_df[
            (ground_truth_df['ImageID'] == image_id)
        ]
        if target_class is not None:
            gt_boxes_df = gt_boxes_df[gt_boxes_df['LabelName'] == target_class]
        
        # Convert ground truth boxes to pixel coordinates
        gt_boxes = []
        for _, row in gt_boxes_df.iterrows():
            gt_box = [
                int(row['XMin'] * width),
                int(row['YMin'] * height),
                int(row['XMax'] * width),
                int(row['YMax'] * height)
            ]
            gt_boxes.append(gt_box)
        
        # Get predictions
        preds = model.predict(image_path, conf=min(confidence_range))
        if len(preds) == 0 or len(preds[0].boxes) == 0:
            return image_results
            
        # Filter predictions for target class
        boxes = preds[0].boxes
        if target_class is not None:
            boxes = [box for box in boxes if int(box.cls) == target_class]
        
        # Process each confidence threshold
        for conf_threshold in confidence_range:
            high_conf_boxes = [box for box in boxes if float(box.conf) >= conf_threshold]
            
            true_positives = 0
            false_positives = 0
            total_iou = 0
            matched_gt = set()
            
            for box in high_conf_boxes:
                pred_box = box.xyxy[0].cpu().numpy()
                best_iou = 0
                best_gt_idx = -1
                
                for i, gt_box in enumerate(gt_boxes):
                    if i in matched_gt:
                        continue
                    iou = calculate_iou(pred_box, gt_box)
                    if iou > best_iou:
                        best_iou = iou
                        best_gt_idx = i
                
                total_iou += best_iou
                
                if best_iou > 0.5:
                    true_positives += 1
                    matched_gt.add(best_gt_idx)
                else:
                    false_positives += 1
            
            # Store results
            image_results[conf_threshold].update({
                'true_positives': true_positives,
                'false_positives': false_positives,
                'false_negatives': len(gt_boxes) - len(matched_gt),
                'total_iou': total_iou,
                'total_predictions': len(high_conf_boxes),
                'confidence_scores': [float(box.conf) for box in high_conf_boxes]
            })
        
        return image_results
        
    except Exception as e:
        print(f"Error processing image {image_path}: {str(e)}")
        return image_results

def analyze_class(model, image_paths, ground_truth_df, confidence_range, target_class):
    """Analyze model predictions for a specific class using sequential processing"""
    print(f"Processing {len(image_paths)} images...")
    
    # Initialize results
    combined_results = {conf: {
        'true_positives': 0,
        'false_positives': 0,
        'false_negatives': 0,
        'total_iou': 0,
        'total_predictions': 0
    } for conf in confidence_range}
    
    confidence_scores = {conf: [] for conf in confidence_range}
    
    # Process images sequentially
    for i, image_path in enumerate(image_paths, 1):
        print(f"Processing image {i}/{len(image_paths)}: {os.path.basename(image_path)}")
        
        try:
            # Get predictions
            preds = model.predict(image_path, conf=min(confidence_range))
            
            # Get image dimensions
            img = cv2.imread(image_path)
            if img is None:
                print(f"Failed to load image: {image_path}")
                continue
            height, width = img.shape[:2]
            
            # Get ground truth boxes
            image_id = Path(image_path).stem
            gt_boxes_df = ground_truth_df[ground_truth_df['ImageID'] == image_id]
            if target_class is not None:
                gt_boxes_df = gt_boxes_df[gt_boxes_df['LabelName'] == target_class]
            
            # Convert ground truth boxes to pixel coordinates
            gt_boxes = []
            for _, row in gt_boxes_df.iterrows():
                gt_box = [
                    int(row['XMin'] * width),
                    int(row['YMin'] * height),
                    int(row['XMax'] * width),
                    int(row['YMax'] * height)
                ]
                gt_boxes.append(gt_box)
            
            if len(preds) == 0 or len(preds[0].boxes) == 0:
                # No predictions for this image
                for conf in confidence_range:
                    combined_results[conf]['false_negatives'] += len(gt_boxes)
                continue
            
            # Filter predictions for target class if specified
            boxes = preds[0].boxes
            if target_class is not None:
                boxes = [box for box in boxes if int(box.cls) == target_class]
            
            # Process each confidence threshold
            for conf_threshold in confidence_range:
                high_conf_boxes = [box for box in boxes if float(box.conf) >= conf_threshold]
                
                true_positives = 0
                false_positives = 0
                total_iou = 0
                matched_gt = set()
                
                for box in high_conf_boxes:
                    pred_box = box.xyxy[0].cpu().numpy()
                    best_iou = 0
                    best_gt_idx = -1
                    
                    for i, gt_box in enumerate(gt_boxes):
                        if i in matched_gt:
                            continue
                        iou = calculate_iou(pred_box, gt_box)
                        if iou > best_iou:
                            best_iou = iou
                            best_gt_idx = i
                    
                    total_iou += best_iou
                    
                    if best_iou > 0.5:
                        true_positives += 1
                        matched_gt.add(best_gt_idx)
                    else:
                        false_positives += 1
                
                # Update combined results
                combined_results[conf_threshold]['true_positives'] += true_positives
                combined_results[conf_threshold]['false_positives'] += false_positives
                combined_results[conf_threshold]['false_negatives'] += (len(gt_boxes) - len(matched_gt))
                combined_results[conf_threshold]['total_iou'] += total_iou
                combined_results[conf_threshold]['total_predictions'] += len(high_conf_boxes)
                confidence_scores[conf_threshold].extend([float(box.conf) for box in high_conf_boxes])
                
        except Exception as e:
            print(f"Error processing image {image_path}: {str(e)}")
            continue
    
    return combined_results, confidence_scores

def plot_metrics(results, confidence_scores, class_name, save_path):
    """Plot and save metrics with curve fitting"""
    print(f"Plotting metrics for {class_name}...")
    thresholds = sorted(results.keys())
    metrics = {
        'precision': [],
        'recall': [],
        'f1': [],
        'iou': []
    }
    
    # Calculate metrics
    for threshold in thresholds:
        tp = results[threshold]['true_positives']
        fp = results[threshold]['false_positives']
        fn = results[threshold]['false_negatives']
        total_iou = results[threshold]['total_iou']
        num_pred = results[threshold]['total_predictions']
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        iou = total_iou / num_pred if num_pred > 0 else 0
        
        metrics['precision'].append(precision)
        metrics['recall'].append(recall)
        metrics['f1'].append(f1)
        metrics['iou'].append(iou)
    
    # Find optimal confidence threshold
    best_idx = np.argmax(metrics['f1'])
    best_threshold = thresholds[best_idx]
    
    # Create smooth curves using spline interpolation
    x_smooth = np.linspace(min(thresholds), max(thresholds), 300)
    curves = {}
    for metric in metrics:
        spline = make_interp_spline(thresholds, metrics[metric], k=3)
        curves[metric] = spline(x_smooth)
    
    # Plot
    plt.figure(figsize=(10, 6))
    colors = {'precision': 'blue', 'recall': 'red', 'f1': 'green', 'iou': 'purple'}
    
    for metric in metrics:
        plt.plot(thresholds, metrics[metric], f'o', color=colors[metric], alpha=0.5, markersize=4)
        plt.plot(x_smooth, curves[metric], '-', color=colors[metric], label=metric.capitalize())
    
    # Add vertical line at optimal threshold
    plt.axvline(x=best_threshold, color='gray', linestyle='--', alpha=0.5)
    plt.text(best_threshold, 0.5, f'Optimal conf={best_threshold:.3f}',
             rotation=90, verticalalignment='center')
    
    plt.xlabel('Confidence Threshold')
    plt.ylabel('Score')
    plt.title(f'Metrics vs Confidence Threshold - {class_name}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    
    return best_threshold, metrics

def draw_example_images(model, image_paths, ground_truth_df, class_name, 
                       conf_threshold, class_id, save_path):
    """Draw and save example images with predictions"""
    print(f"Creating example visualization for {class_name}...")
    fig, axes = plt.subplots(2, 4, figsize=(15, 8))
    fig.suptitle(f'Example Predictions for {class_name}\n' +
                 f'Confidence Threshold: {conf_threshold:.3f}', fontsize=12)
    
    for idx, (img_path, ax) in enumerate(zip(image_paths[:8], axes.flat)):
        # Load and convert image
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        height, width = img.shape[:2]
        
        # Get predictions
        results = model.predict(img_path, conf=conf_threshold)
        
        # Display image
        ax.imshow(img)
        
        # Draw predicted boxes
        if len(results) > 0:
            boxes = results[0].boxes
            for box in boxes:
                if int(box.cls) == class_id:  # Only draw boxes for target class
                    conf = float(box.conf)
                    x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
                    rect = plt.Rectangle((x1, y1), x2-x1, y2-y1,
                                      fill=False, color='red', linewidth=1)
                    ax.add_patch(rect)
                    ax.text(x1, y1-5, f'{conf:.2f}', color='red', fontsize=8)
        
        ax.set_axis_off()
        ax.set_title(f'Image: {Path(img_path).stem}', fontsize=8)
    
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def create_yolo_dataset_config(output_dir, class_descriptions, annotations_df, image_dir, class_image_dict=None):
    """Create YOLO format dataset configuration and label files"""
    print("Creating YOLO format dataset configuration...")
    
    # Create directories
    dataset_dir = os.path.join(output_dir, 'yolo_validation')
    images_dir = os.path.join(dataset_dir, 'images')
    labels_dir = os.path.join(dataset_dir, 'labels')
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)
    
    # Create class mapping - simpler version using DataFrame index
    class_map = {row['LabelName']: idx for idx, (_, row) in enumerate(class_descriptions.iterrows())}
    
    # Print class mapping for verification
    print("Class mapping:")
    for label, idx in class_map.items():
        print(f"{label}: {idx}")
    
    # Save class names file
    names_file = os.path.join(dataset_dir, 'classes.txt')
    with open(names_file, 'w') as f:
        for _, row in class_descriptions.iterrows():
            f.write(f"{row['DisplayName']}\n")
    
    # Filter images if class_image_dict is provided
    if class_image_dict is not None:
        valid_images = set().union(*class_image_dict.values())
        annotations_df = annotations_df[annotations_df['ImageID'].isin(valid_images)]
    
    # Process each unique image
    processed_images = set()
    for image_id in annotations_df['ImageID'].unique():
        if image_id in processed_images:
            continue
            
        # Source image path
        source_img_path = os.path.join(image_dir, f"{image_id}.jpg")
        if not os.path.exists(source_img_path):
            print(f"Warning: Image not found: {source_img_path}")
            continue
            
        # Create symbolic link to image
        dest_img_path = os.path.join(images_dir, f"{image_id}.jpg")
        if os.path.exists(dest_img_path):
            os.remove(dest_img_path)
        try:
            os.symlink(os.path.abspath(source_img_path), dest_img_path)
        except OSError as e:
            print(f"Warning: Could not create symlink, copying file instead: {e}")
            shutil.copy2(source_img_path, dest_img_path)
        
        # Get image dimensions
        img = cv2.imread(source_img_path)
        if img is None:
            print(f"Warning: Could not read image: {source_img_path}")
            continue
        height, width = img.shape[:2]
        
        # Create YOLO format label file
        img_annotations = annotations_df[annotations_df['ImageID'] == image_id]
        label_file = os.path.join(labels_dir, f"{image_id}.txt")
        
        with open(label_file, 'w') as f:
            for _, ann in img_annotations.iterrows():
                # Convert bbox to YOLO format (normalized coordinates)
                x_center = (ann['XMin'] + ann['XMax']) / 2
                y_center = (ann['YMin'] + ann['YMax']) / 2
                bbox_width = ann['XMax'] - ann['XMin']
                bbox_height = ann['YMax'] - ann['YMin']
                
                # Get class index
                try:
                    class_idx = class_map[ann['LabelName']]
                except KeyError:
                    print(f"Warning: Unknown class label {ann['LabelName']} for image {image_id}")
                    continue
                
                # Write YOLO format line: <class_idx> <x_center> <y_center> <width> <height>
                f.write(f"{class_idx} {x_center} {y_center} {bbox_width} {bbox_height}\n")
        
        processed_images.add(image_id)
    
    # Create dataset YAML configuration
    yaml_content = {
        'path': os.path.abspath(dataset_dir),
        'train': 'images',  # Not used but required
        'val': 'images',
        'names': {idx: row['DisplayName'] for idx, (_, row) in enumerate(class_descriptions.iterrows())}
    }
    
    yaml_path = os.path.join(dataset_dir, 'dataset.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, sort_keys=False)
    
    print(f"Created YOLO dataset configuration in {dataset_dir}")
    print(f"Processed {len(processed_images)} images")
    print(f"Class names in YAML:")
    for idx, name in yaml_content['names'].items():
        print(f"{idx}: {name}")
    
    return yaml_path

def calculate_class_map50(model, yaml_path, class_idx):
    """Calculate mAP@50 for a specific class using YOLO's validator"""
    try:
        # Run validation for specific class
        metrics = model.val(data=yaml_path, split='val', classes=[class_idx])
        return metrics[0].maps  # Get mAP@50 for the class
    except Exception as e:
        print(f"Error calculating mAP@50: {str(e)}")
        return 0.0

In [12]:
print("Starting model evaluation...")

# Set random seed for reproducibility
np.random.seed(42)

# Create output directory
output_dir = create_output_folder()

# Load class descriptions
print("\nLoading class descriptions...")
class_desc_path = 'oidv7-class-descriptions-boxable.csv'
class_descriptions = load_class_descriptions(class_desc_path)
print(f"Found {len(class_descriptions)} classes to evaluate")

# Load annotations
print("\nLoading and filtering annotations...")
annotations_path = 'test-annotations-bbox.csv'
image_dir = 'image_data/test_images_25_class'

annotations_df, class_image_dict = load_and_filter_annotations(
    annotations_path, class_descriptions, image_dir, images_per_class=200)
print(f"Found {len(annotations_df)} relevant annotations")

# Load model
print("\nLoading YOLO model...")
model_path = 'E:/Data 255/YOLO multi-class/yolo_training/multi_class_run_4000_images_20241121_190831/yolo_boxes_4000/weights/best.pt'
model = YOLO(model_path)

# Create confidence range
confidence_range = create_confidence_range()
print(f"Created confidence range with {len(confidence_range)} thresholds")

# Create results dataframe
results_df = pd.DataFrame(columns=[
    'DisplayName', 'mAP@50', 'optimal_conf', 'f1', 'precision',
    'recall', 'mean_IoU', 'mAP'
])

# Process each class
print("\nProcessing individual classes...")
total_classes = len(class_descriptions)
yaml_path = create_yolo_dataset_config(
    output_dir, class_descriptions, annotations_df, image_dir, class_image_dict)

# Process each class
for idx, (_, class_row) in enumerate(class_descriptions.iterrows(), 1):
    class_name = class_row['DisplayName']
    class_id = class_row['LabelName']
    
    print(f"\nProcessing class {idx}/{total_classes}: {class_name}")
    
    # Get selected images for this class
    class_images = class_image_dict[class_id]
    image_paths = [
        os.path.join(image_dir, f"{img_id}.jpg")
        for img_id in class_images
    ]
    
    print(f"Using {len(image_paths)} images for {class_name}")
    
    # Analyze class
    results, confidence_scores = analyze_class(
        model, image_paths, annotations_df, confidence_range, class_id)
    
    # Plot metrics and get optimal threshold
    metrics_path = os.path.join(output_dir, f'metrics_{class_name}.png')
    opt_threshold, metrics = plot_metrics(
        results, confidence_scores, class_name, metrics_path)
    
    # Draw example images
    examples_path = os.path.join(output_dir, f'examples_{class_name}.png')
    draw_example_images(
        model, image_paths, annotations_df, class_name,
        opt_threshold, class_id, examples_path)
    
    # Calculate mAP@50
    print(f"Calculating mAP@50 for {class_name}...")
    map50 = calculate_class_map50(model, yaml_path, idx)
    
    # Add to results dataframe
    best_idx = np.argmax(metrics['f1'])
    results_df = pd.concat([results_df, pd.DataFrame([{
        'DisplayName': class_name,
        'mAP@50': map50,
        'optimal_conf': opt_threshold,
        'f1': metrics['f1'][best_idx],
        'precision': metrics['precision'][best_idx],
        'recall': metrics['recall'][best_idx],
        'mean_IoU': metrics['iou'][best_idx],
        'mAP': map50,
        'num_test_images': len(image_paths)
    }])], ignore_index=True)

# Calculate overall mAP@50
print("\nCalculating overall mAP@50...")
overall_metrics = model.val(data=yaml_path, split='val')
overall_map50 = overall_metrics[0].maps

# Save results to CSV
csv_path = os.path.join(output_dir, 'model_evaluation_results.csv')
results_df.to_csv(csv_path, index=False)

# Save selected image IDs for reproducibility
image_ids_df = pd.DataFrame({
    'class_id': [k for k, v in class_image_dict.items() for _ in v],
    'image_id': [img_id for v in class_image_dict.values() for img_id in v]
})
image_ids_df.to_csv(os.path.join(output_dir, 'selected_test_images.csv'), index=False)

# Print summary
print("\nModel Evaluation Results:")
print(results_df.to_string(index=False))

print(f"\nResults have been saved to: {output_dir}")
print("Files saved:")
print(f"- {csv_path} (metrics for all classes)")
print("- selected_test_images.csv (list of images used for testing)")
print("- metrics_*.png (confidence curves)")
print("- examples_*.png (example predictions)")

Starting model evaluation...

Created output directory: model_evaluation_results_20241130_162103

Loading class descriptions...
Found 25 classes to evaluate

Loading and filtering annotations...
Loading annotations...
Selected 4031 unique images across all classes
Class /m/09j5n: 200 images
Class /m/01xyhv: 200 images
Class /m/0jyfg: 200 images
Class /m/01d40f: 200 images
Class /m/0fly7: 200 images
Class /m/0h9mv: 200 images
Class /m/0463sg: 200 images
Class /m/0hg7b: 200 images
Class /m/0342h: 200 images
Class /m/0138tl: 200 images
Class /m/01n5jq: 200 images
Class /m/0271t: 200 images
Class /m/01bqk0: 200 images
Class /m/0c_jw: 200 images
Class /m/02dgv: 200 images
Class /m/032b3c: 200 images
Class /m/052lwg6: 200 images
Class /m/01y9k5: 200 images
Class /m/02dl1y: 200 images
Class /m/03120: 200 images
Class /m/01bfm9: 200 images
Class /m/04szw: 200 images
Class /m/01n4qj: 200 images
Class /m/0dv5r: 200 images
Class /m/01599: 200 images
Found 11661 relevant annotations

Loading YOLO 

C:\Users\User\AppData\Local\Temp\ipykernel_58076\3597823495.py:359: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()



image 1/1 e:\Data 255\YOLO multi-class\image_data\test_images_25_class\055aa9c8e194b5a4.jpg: 640x480 45 Footwears, 1 Tire, 6 Fashion accessorys, 2 Toys, 3 Bicycle wheels, 1 Hat, 12 Shortss, 2 Musical instruments, 11.0ms
Speed: 3.0ms preprocess, 11.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 e:\Data 255\YOLO multi-class\image_data\test_images_25_class\05c77dd04223955b.jpg: 480x640 25 Footwears, 2 Glassess, 2 Fashion accessorys, 2 Microphones, 2 Toys, 1 Bicycle wheel, 3 Hats, 9 Shortss, 1 Musical instrument, 11.0ms
Speed: 2.0ms preprocess, 11.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 e:\Data 255\YOLO multi-class\image_data\test_images_25_class\06aa36fae1159215.jpg: 640x640 10 Footwears, 8 Dresss, 19 Fashion accessorys, 3 Posters, 1 Shorts, 10.0ms
Speed: 4.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 e:\Data 255\YOLO multi-class\image_data\test_images_25_class\0

val: Scanning E:\Data 255\YOLO multi-class\model_evaluation_results_20241130_162103\yolo_validation\labels... 4031 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4031/4031 [00:02<00:00, 1518.04it/s]


val: New cache created: E:\Data 255\YOLO multi-class\model_evaluation_results_20241130_162103\yolo_validation\labels.cache


...

                   all       4031        461      0.568       0.43      0.463      0.332
                 Dress        246        461      0.568       0.43      0.463      0.332
Speed: 0.3ms preprocess, 2.3ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to c:\Users\User\runs\detect\val6
Error calculating mAP@50: 'DetMetrics' object is not subscriptable

Processing class 4/25: Dress
Using 200 images for Dress
Processing 200 images...
Processing image 1/200: 124724830005d74a.jpg

image 1/1 e:\Data 255\YOLO multi-class\image_data\test_images_25_class\124724830005d74a.jpg: 320x640 5 Footwears, 8 Dresss, 77 Fashion accessorys, 1 Poster, 3 Furnitures, 55.0ms
Speed: 3.0ms preprocess, 55.0ms inference, 6.0ms postprocess per image at shape (1, 3, 320, 640)
Processing image 2/200: 512bba868790ca67.jpg

image 1/1 e:\Data 255\YOLO multi-class\image_data\test_images_25_class\512bba868790ca67.jpg: 640x512 1 Footwear, 5 Dresss, 1 Jeans, 46 Fashion accessorys, 10 Furnitures, 1 Door

C:\Users\User\AppData\Local\Temp\ipykernel_58076\3597823495.py:359: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()



image 1/1 e:\Data 255\YOLO multi-class\image_data\test_images_25_class\124724830005d74a.jpg: 320x640 5 Footwears, 8 Dresss, 77 Fashion accessorys, 1 Poster, 3 Furnitures, 15.0ms
Speed: 2.0ms preprocess, 15.0ms inference, 6.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 e:\Data 255\YOLO multi-class\image_data\test_images_25_class\512bba868790ca67.jpg: 640x512 1 Footwear, 5 Dresss, 1 Jeans, 46 Fashion accessorys, 10 Furnitures, 1 Door, 1 Jacket, 15.0ms
Speed: 3.0ms preprocess, 15.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 e:\Data 255\YOLO multi-class\image_data\test_images_25_class\1928189562dc619a.jpg: 448x640 28 Fashion accessorys, 22.0ms
Speed: 2.0ms preprocess, 22.0ms inference, 3.0ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 e:\Data 255\YOLO multi-class\image_data\test_images_25_class\0812c925577193d4.jpg: 640x512 75 Footwears, 5 Suits, 7 Dresss, 30 Jeanss, 54 Fashion accessorys, 3 Bicycle wheels, 4 Furnitures, 1

val: Scanning E:\Data 255\YOLO multi-class\model_evaluation_results_20241130_162103\yolo_validation\labels.cache... 4031 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4031/4031 [00:00<?, ?it/s]


KeyboardInterrupt: 